In [2]:
from pathlib import Path
from collections import Counter
from PIL import Image, ImageOps
from tqdm import tqdm
import sys
from itertools import chain

In [3]:
def scan_resolutions(src_dir: Path):
    sizes = Counter()
    img_paths = chain(src_dir.rglob("*.png"), src_dir.rglob("*.BMP"))

    for img in img_paths:
        try:
            with Image.open(img) as im:
                sizes[im.size] += 1
        except Exception as e:
            print(f"[WARN] {img}: {e}", file=sys.stderr)

    return sizes

In [7]:
def resize_dataset(src_dir: Path, dst_dir: Path, target: int, keep_ratio=True):
    dst_dir.mkdir(parents=True, exist_ok=True)
    img_paths = chain(src_dir.rglob("*.png"), src_dir.rglob("*.BMP"))

    for img in tqdm(list(img_paths), desc="Resizing"):
        rel_path  = img.relative_to(src_dir)
        save_path = dst_dir / rel_path
        save_path.parent.mkdir(parents=True, exist_ok=True)

        try:
            with Image.open(img) as im:
                if keep_ratio:
                    pad_color = _black_for_mode(im.mode)
                    im = ImageOps.pad(
                        im, (target, target),
                        color=pad_color,
                        centering=(0.5, 0.5),
                        method=Image.Resampling.LANCZOS,
                    )
                else:
                    im = im.resize((target, target), Image.Resampling.LANCZOS)

                im.save(save_path)
        except Exception as e:
            print(f"[WARN] {img}: {e}", file=sys.stderr)       

In [8]:
def _black_for_mode(mode: str):
    """이미지 모드에 맞는 '검정' 값을 반환한다."""
    if mode in ("RGB", "RGBA"):
        return (0, 0, 0)           # 3채널 / 4채널
    elif mode in ("L", "I", "F"):
        return 0                   # 단일 채널
    else:
        # 그 외 팔레트(P) 등은 흑백으로 변환 후 처리
        return 0

In [9]:
SRC_DIR = Path('blast/split')
DST_DIR = Path('blast/split_vit224')
TARGET = 224

In [10]:
res_counter = scan_resolutions(SRC_DIR)

print("\n[해상도 분포]")
for (w, h), cnt in sorted(res_counter.items(), key=lambda x: -x[1]):
    print(f"  {w:4d}×{h:4d} : {cnt}장")

# --- 리사이즈 필요 여부 판단 ---
all_same_target = (
    len(res_counter) == 1 and
    list(res_counter.keys())[0] == (TARGET, TARGET)
)

if all_same_target:
    print("\n모든 이미지가 이미 목표 해상도입니다. 리사이즈를 건너뜁니다.")
else:
    # 필요 시 yes/no 입력받기
    proceed = input(f"\n{TARGET}×{TARGET} 로 리사이즈 후 {DST_DIR} 에 저장할까요? [y/N] ")
    if proceed.lower() == "y":
        print(f"\n[2] 리사이즈 시작…")
        resize_dataset(SRC_DIR, DST_DIR, TARGET, keep_ratio=True)
        print("✅ 완료!")
    else:
        print("⏩ 리사이즈를 취소했습니다.")


[해상도 분포]
   512× 384 : 641장
   468× 406 : 2장
   490× 420 : 1장
   442× 424 : 1장
   469× 389 : 1장
   455× 394 : 1장
   484× 420 : 1장
   388× 321 : 1장
   401× 356 : 1장
   469× 403 : 1장
   472× 423 : 1장
   465× 395 : 1장
   480× 416 : 1장
   437× 382 : 1장
   423× 410 : 1장
   464× 362 : 1장
   388× 344 : 1장
   506× 406 : 1장
   454× 383 : 1장
   458× 419 : 1장
   486× 388 : 1장
   445× 360 : 1장
   454× 418 : 1장
   424× 361 : 1장
   470× 424 : 1장
   427× 379 : 1장
   511× 424 : 1장
   526× 426 : 1장
   485× 384 : 1장
   483× 443 : 1장
   425× 415 : 1장
   469× 414 : 1장
   466× 400 : 1장
   458× 416 : 1장
   505× 398 : 1장
   443× 432 : 1장
   469× 426 : 1장
   389× 382 : 1장
   440× 398 : 1장
   469× 393 : 1장
   498× 430 : 1장
   471× 384 : 1장
   425× 373 : 1장
   441× 402 : 1장
   438× 373 : 1장
   473× 412 : 1장
   517× 351 : 1장
   433× 394 : 1장
   494× 418 : 1장
   482× 442 : 1장
   497× 425 : 1장
   487× 402 : 1장
   478× 413 : 1장
   520× 459 : 1장
   417× 379 : 1장
   474× 391 : 1장
   397× 305 : 1장
   435× 402 : 1장
  

Resizing: 100%|██████████| 812/812 [00:18<00:00, 44.23it/s] 

✅ 완료!
